In [1]:
from PIL import Image
import numpy as np
import os

In [2]:
test_dir = "/Users/sasha/Documents/fetal-tumor-segmentation/data/dataset/test/filled_masks"
pred_dir = "/Users/sasha/Documents/fetal-tumor-segmentation/results/predictions"

In [3]:
def dice_score(pred, target, eps=1e-6):

    intersection = np.sum(pred * target)
    total = np.sum(pred) + np.sum(target)

    if total == 0:   
        return 1.0

    return (2.0 * intersection + eps) / (total + eps)


def iou_score(pred, target, eps=1e-6):
    
    intersection = np.sum(pred * target)
    union = np.sum(pred) + np.sum(target) - intersection

    if union == 0:   
        return 1.0

    return (intersection + eps) / (union + eps)

In [4]:
def load_mask(path):
    img = Image.open(path).convert("L")  
    mask = np.array(img)

    mask = (mask > 0).astype(np.float32)

    return mask

In [5]:
dice_scores = []
iou_scores = []

gt_files = [f for f in os.listdir(test_dir) if f.endswith("_Annotation.png")]

print(f"Found {len(gt_files)} ground truth masks\n")

for gt_name in sorted(gt_files):

    base_id = gt_name.replace("_Annotation.png", "")
    pred_name = f"{base_id}_pred.png"

    gt_path = os.path.join(test_dir, gt_name)
    pred_path = os.path.join(pred_dir, pred_name)

    if not os.path.exists(pred_path):
        print(f"Missing prediction for {gt_name}")
        continue

    gt_mask = load_mask(gt_path)
    pred_mask = load_mask(pred_path)

    if gt_mask.shape != pred_mask.shape:
        print(f"Shape mismatch for {base_id}")
        continue

    d = dice_score(pred_mask, gt_mask)
    i = iou_score(pred_mask, gt_mask)

    dice_scores.append(d)
    iou_scores.append(i)

    print(f"{base_id:20s} | Dice: {d:.4f} | IoU: {i:.4f}")

Found 101 ground truth masks

100_2HC              | Dice: 0.9487 | IoU: 0.9024
10_2HC               | Dice: 0.5121 | IoU: 0.3442
114_2HC              | Dice: 0.9446 | IoU: 0.8950
125_2HC              | Dice: 0.8732 | IoU: 0.7749
132_HC               | Dice: 0.9566 | IoU: 0.9169
152_2HC              | Dice: 0.9515 | IoU: 0.9075
156_HC               | Dice: 0.9471 | IoU: 0.8996
185_HC               | Dice: 0.9534 | IoU: 0.9110
186_HC               | Dice: 0.9465 | IoU: 0.8985
192_HC               | Dice: 0.9703 | IoU: 0.9423
194_HC               | Dice: 0.9405 | IoU: 0.8876
196_HC               | Dice: 0.9065 | IoU: 0.8290
1_HC                 | Dice: 0.9156 | IoU: 0.8444
204_HC               | Dice: 0.9339 | IoU: 0.8760
205_HC               | Dice: 0.9697 | IoU: 0.9412
21_2HC               | Dice: 0.8751 | IoU: 0.7780
241_HC               | Dice: 0.9625 | IoU: 0.9277
249_HC               | Dice: 0.9799 | IoU: 0.9606
261_HC               | Dice: 0.9728 | IoU: 0.9470
287_2HC             

In [6]:
print(f"Mean Dice : {np.mean(dice_scores):.4f}")
print(f"Mean IoU  : {np.mean(iou_scores):.4f}")

Mean Dice : 0.9367
Mean IoU  : 0.9000
